## User TimeLine Analysing

In [3]:
import tweepy
import pandas as pd

In [4]:
app_api_key = '-'
app_api_secret_key = '-'


auth = tweepy.AppAuthHandler(app_api_key, app_api_secret_key)

In [5]:
def get_tweets(username: str, count: int = 1000):
    '''get tweets tweeted by specific user'''
    api = tweepy.API(
            auth,
            wait_on_rate_limit=True
        )

    tweets = tweepy.Cursor(
            api.user_timeline,
            screen_name = username,
            tweet_mode = 'extended',
            count = count
        ).items()
    
    tweets = [tweet._json for tweet in tweets]
    df = pd.json_normalize(tweets)

    return df


In [6]:
df = get_tweets('alluarjun')